In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import packages.preprocess.dataframe as df_module

df = df_module.trail_dataframe()
df.head()

ModuleNotFoundError: No module named 'packages'

NameError: name '__file__' is not defined